#### install MOFA if necessary

In [1]:
version

               _                           
platform       x86_64-pc-linux-gnu         
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          0.2                         
year           2020                        
month          06                          
day            22                          
svn rev        78730                       
language       R                           
version.string R version 4.0.2 (2020-06-22)
nickname       Taking Off Again            

### requirements

Either install libraries mentioned below yourself or run the following docker command in the directory with this jupyter notebook: <br> `docker run -it --rm -p 8888:8888 -v $(pwd):/home/rstudio jonasjonker/mofa2-tutorial:0.0.1`

### [tutorial](https://raw.githack.com/bioFAM/MOFA2_tutorials/master/R_tutorials/10x_scRNA_scATAC.html)
Vignette demonstrating how MOFA can be used to integrate scRNA-seq and scATAC-seq data.

### [JASPAR2020](http://jaspar.genereg.net/)
The JASPAR CORE contains a curated, non-redundant set of profiles, derived from published and experimentally defined transcription factor binding sites for eukaryotes. It should be used, when seeking models for specific factors or structural classes, or if experimental evidence is paramount. 

### [BSgenome.Hsapiens.UCSC.hg38](http://bioconductor.org/packages/release/data/annotation/html/BSgenome.Hsapiens.UCSC.hg38.html)
Full genome sequences for Homo sapiens (Human) as provided by UCSC (hg38, based on GRCh38.p12) and stored in Biostrings objects.

In [3]:
# BiocManager::install("motifmatchr") # <-- not installed in the docker image

library(data.table)
library(ggplot2)
library(magrittr)
library(Seurat)
library(Signac)
library(motifmatchr)

# for GSEA analysis
library(msigdbr)

# For motif enrichment analysis
library(JASPAR2020)
library(TFBSTools)
library(BSgenome.Hsapiens.UCSC.hg38)

# MOFA
library(MOFA2)

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, w

### load R data object

In [3]:
# seurat <- readRDS(url("ftp://ftp.ebi.ac.uk/pub/databases/mofa/10x_rna_atac_vignette/seurat.rds"))
seurat <- readRDS('seurat.rds')  # from disk
seurat  

An object of class Seurat 
138109 features across 11909 samples within 2 assays 
Active assay: RNA (29732 features, 0 variable features)
 1 other assay present: ATAC

### Inspect seurat.rds
[Seurat]("https://satijalab.org/seurat/") is an R package designed for QC, analysis, and exploration of single-cell RNA-seq data.

In [5]:
str(seurat)

Formal class 'Seurat' [package "Seurat"] with 13 slots


Warning message:
"Not a validObject(): no slot of name "images" for this object of class "Seurat""


  ..@ assays      :List of 2
  .. ..$ RNA :Formal class 'Assay' [package "Seurat"] with 8 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:23508033] 9 36 41 47 72 91 95 107 127 129 ...
  .. .. .. .. .. ..@ p       : int [1:11910] 0 3308 5204 8108 8954 11236 12589 15650 17341 20369 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 29732 11909
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:29732] "AL627309.1" "AL627309.2" "AL627309.5" "AL627309.4" ...
  .. .. .. .. .. .. ..$ : chr [1:11909] "AAACAGCCAAGGAATC" "AAACAGCCAATCCCTT" "AAACAGCCAATGCGCT" "AAACAGCCACACTAAT" ...
  .. .. .. .. .. ..@ x       : num [1:23508033] 1 1 1 1 1 1 1 1 1 6 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:23508033] 9 36 41 47 72 91 95 107 127 129 ...
  .. .. .. .. .. ..@ p       : int [1:11910]

In [11]:
head (seurat@meta.data, 1)
head(seurat@meta.data[,c("celltype","broad_celltype","pass_rnaQC","pass_accQC")], 1)
table(seurat@meta.data$celltype)
table(seurat@meta.data$broad_celltype)

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ATAC,nFeature_ATAC,celltype,pass_rnaQC,pass_accQC,broad_celltype
,<fct>,<dbl>,<int>,<dbl>,<int>,<chr>,<lgl>,<lgl>,<chr>
AAACAGCCAAGGAATC,scRNA+scATAC PBMC,8380,3308,55582,13878,naive CD4 T cells,TRUE,TRUE,Lymphoid


,celltype,broad_celltype,pass_rnaQC,pass_accQC
,<chr>,<chr>,<lgl>,<lgl>
AAACAGCCAAGGAATC,naive CD4 T cells,Lymphoid,TRUE,TRUE



 CD56 (bright) NK cells     CD56 (dim) NK cells     classical monocytes 
                    407                     472                    1929 
   effector CD8 T cells  intermediate monocytes            MAIT T cells 
                    385                     664                     106 
         memory B cells      memory CD4 T cells              myeloid DC 
                    420                    1611                     242 
          naive B cells       naive CD4 T cells       naive CD8 T cells 
                    295                    1462                    1549 
non-classical monocytes         plasmacytoid DC 
                    383                     107 


Lymphoid  Myeloid 
    6814     3218 

### filter low quality data

In [5]:
seurat
seurat <- seurat %>%
  .[,seurat@meta.data$pass_accQC==TRUE & seurat@meta.data$pass_rnaQC==TRUE]  # filter data
seurat

An object of class Seurat 
138109 features across 11909 samples within 2 assays 
Active assay: RNA (29732 features, 0 variable features)
 1 other assay present: ATAC

An object of class Seurat 
138109 features across 10032 samples within 2 assays 
Active assay: RNA (29732 features, 0 variable features)
 1 other assay present: ATAC

In [14]:
seurat@assays$RNA
seurat@assays$ATAC

Assay data with 29732 features for 11909 cells
First 10 features:
 AL627309.1, AL627309.2, AL627309.5, AL627309.4, AP006222.2, AL669831.2,
LINC01409, FAM87B, LINC01128, LINC00115 

Assay data with 108377 features for 11909 cells
First 10 features:
 chr1:10109-10357, chr1:180730-181630, chr1:191491-191736,
chr1:267816-268196, chr1:586028-586373, chr1:629721-630172,
chr1:633793-634264, chr1:777634-779926, chr1:816881-817647,
chr1:819912-823500 

### Load and attach metadata

In [15]:
feature_metadata <- fread("ftp://ftp.ebi.ac.uk/pub/databases/mofa/10x_rna_atac_vignette/filtered_feature_bc_matrix/features.tsv.gz") %>%
  setnames(c("ens_id","gene","view","chr","start","end"))

In [16]:
feature_metadata.rna <- feature_metadata[view=="Gene Expression"]
head(feature_metadata.rna,n=3)

ens_id,gene,view,chr,start,end
<chr>,<chr>,<chr>,<chr>,<int>,<int>
ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267
ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
ENSG00000186092,OR4F5,Gene Expression,chr1,65418,69055


In [17]:
feature_metadata.atac <-
    feature_metadata[view=="Peaks"] %>%  # Filter Peaks
    .[,ens_id:=NULL] %>%                 # Update their ens_i to NULL
    setnames("gene","peak")              # rename column gene to peak
head(feature_metadata.atac,n=3)
table(feature_metadata.atac$view)

peak,view,chr,start,end
<chr>,<chr>,<chr>,<int>,<int>
chr1:10109-10357,Peaks,chr1,10109,10357
chr1:180730-181630,Peaks,chr1,180730,181630
chr1:191491-191736,Peaks,chr1,191491,191736



 Peaks 
108377 

In [18]:
foo <- fread("ftp://ftp.ebi.ac.uk/pub/databases/mofa/10x_rna_atac_vignette/atac_peak_annotation.tsv") %>%
  .[,c("peak","peak_type")] %>%
  .[peak_type%in%c("distal", "promoter")]

feature_metadata.atac <- feature_metadata.atac %>% 
  merge(foo,by="peak",all.x=TRUE)

head(feature_metadata.atac, n=3)

table(feature_metadata.atac$peak_type)
table(feature_metadata.atac$view)

peak,view,chr,start,end,peak_type
<chr>,<chr>,<chr>,<int>,<int>,<chr>
GL000194.1:101218-101619,Peaks,GL000194.1,101218,101619,distal
GL000194.1:114724-115221,Peaks,GL000194.1,114724,115221,promoter
GL000194.1:28230-28519,Peaks,GL000194.1,28230,28519,distal



  distal promoter 
   91822    16074 


 Peaks 
108377 

### create motif matrix
[Create motif matrix](https://satijalab.org/signac/reference/CreateMotifMatrix.html)

In [19]:
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(species = "9606", all_versions = FALSE, matrixtype = "PFM")  ## matrixtype = "PWM" is also fine for our purposes
)
pfm

PFMatrixList of length 633
names(633): MA0030.1 MA0031.1 MA0051.1 MA0057.1 ... MA0748.2 MA0528.2 MA0609.2

#### know thy data

In [15]:
pfm$`MA0030.1`

An object of class PFMatrix
ID: MA0030.1
Name: FOXF2
Matrix Class: Fork head / winged helix factors
strand: +
Tags: 
$alias
[1] "FREAC2"

$description
[1] "forkhead box F2"

$family
[1] "Forkhead box (FOX) factors"

$medline
[1] "7957066"

$symbol
[1] "FOXF2"

$tax_group
[1] "vertebrates"

$tfbs_shape_id
[1] "37"

$type
[1] "SELEX"

$collection
[1] "CORE"

$species
          9606 
"Homo sapiens" 

$acc
[1] "Q12947"

Background: 
   A    C    G    T 
0.25 0.25 0.25 0.25 
Matrix: 
  [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
A    1   10   17   13    3    7    0   27   27    27     0    27    16     7
C   10    7    4    5   11    0    0    0    0     0    25     0     4     4
G    7    5    2    5    8   20    0    0    0     0     0     0     2     6
T    9    5    4    5    0    0   27    0    0     0     2     0     5    10

In [16]:
# BSgenome.Hsapiens.UCSC.hg38 contains full chromosomes and contigs.
BSgenome.Hsapiens.UCSC.hg38$chr1_GL383518v1_alt


182439-letter DNAString object
seq: CTATAATCCCAGCTTGTTGGGAGGCCAAGGCAGGAG...TAATCAGACTAAAAATACCAGAGGCAATTTGAATTC

### test creation of GRanges object

In [21]:
peaks.granges <- feature_metadata.atac %>%
    .[peak_type=="distal"] %>%
    makeGRangesFromDataFrame(keep.extra.columns = TRUE, ignore.strand = TRUE)

In [22]:
str(peaks.granges)
table(peaks.granges@seqnames)

Formal class 'GRanges' [package "GenomicRanges"] with 7 slots
  ..@ seqnames       :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 31 levels "chr1","chr2",..: 24 25 26 27 28 29 30 31 10 11 ...
  .. .. ..@ lengths        : int [1:31] 3 2 6 10 6 1 1 1 4324 4380 ...
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ ranges         :Formal class 'IRanges' [package "IRanges"] with 6 slots
  .. .. ..@ start          : int [1:91822] 101218 28230 56140 24252 30140 140550 62326 67888 70583 87914 ...
  .. .. ..@ width          : int [1:91822] 402 290 52 26 3216 466 2345 1742 286 370 ...
  .. .. ..@ NAMES          : NULL
  .. .. ..@ elementType    : chr "ANY"
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  ..@ strand         :Formal class 'Rle' [package "S4Vectors"] with 4 slots
  .. .. ..@ values         : Factor w/ 3 levels "+","-","*": 3
  .. .. ..@ lengths        : int 91822
  .. .. ..@ elementMetad


      chr1       chr2       chr3       chr4       chr5       chr6       chr7 
      8861       7382       5877       3653       4587       5772       4475 
      chr8       chr9      chr10      chr11      chr12      chr13      chr14 
      3827       3731       4324       4380       4813       2079       3131 
     chr15      chr16      chr17      chr18      chr19      chr20      chr21 
      2994       3354       4805       1776       3751       2686       1177 
     chr22       chrX GL000194.1 GL000195.1 GL000205.2 GL000219.1 KI270713.1 
      2014       2343          3          2          6         10          6 
KI270726.1 KI270727.1 KI270734.1 
         1          1          1 

### Remove unlocalized contigs

In [24]:
peaks.granges <- keepStandardChromosomes(peaks.granges, pruning.mode="coarse")
table(peaks.granges@seqnames)


 chr1  chr2  chr3  chr4  chr5  chr6  chr7  chr8  chr9 chr10 chr11 chr12 chr13 
 8861  7382  5877  3653  4587  5772  4475  3827  3731  4324  4380  4813  2079 
chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22  chrX 
 3131  2994  3354  4805  1776  3751  2686  1177  2014  2343 

### generate GRanges and motif matrix
this step takes some time.

In [25]:
for (i in c("distal","promoter")) {
  peaks.granges <- feature_metadata.atac %>%
    .[peak_type==i] %>%
    .[,c("chr","start","end","peak")] %>%
    makeGRangesFromDataFrame(keep.extra.columns = TRUE, ignore.strand = TRUE)
    
  # prevents error due to different naming conventions of unlocalized contigs by removing them.
  peaks.granges <- keepStandardChromosomes(peaks.granges, pruning.mode="coarse")
    
  # Scan motifs throughout the DNA sequence of each peak and create a binary matrix of motif-peak presence.
  motif.matrix <- Signac::CreateMotifMatrix(
    features = peaks.granges,               # GRanges object containing a set of genomic features
    pwm = pfm,                              # PFMatrixList/PWMatrixList object containing position weight/frequency matrices
    genome = BSgenome.Hsapiens.UCSC.hg38,   # Object compatible with genome argument in motifmatchr::matchMotifs
    use.counts = FALSE                      # default
  ) %>% as.matrix
  
  # AddChromatinAssay to the Seurat object
  seurat@assays[[paste0("ATAC_",i)]] <- CreateChromatinAssay(
    seurat@assays$ATAC@counts[peaks.granges$peak,], 
    ranges = peaks.granges,
    motifs = CreateMotifObject(motif.matrix, pfm)
  )
  
}
# saveRDS(seurat, "seurat_with_motif_matrix.rds")
seurat

Warning message in rownames(x = object) == rownames(x = new.data):
"longer object length is not a multiple of shorter object length"
Warning message in SetAssayData.ChromatinAssay(object = new.assay, slot = "motifs", :
"Features do not match in ChromatinAssay and Motif object.
                Subsetting the Motif object."
Warning message in rownames(x = object) == rownames(x = new.data):
"longer object length is not a multiple of shorter object length"
Warning message in SetAssayData.ChromatinAssay(object = new.assay, slot = "motifs", :
"Features do not match in ChromatinAssay and Motif object.
                Subsetting the Motif object."


An object of class Seurat 
245458 features across 11909 samples within 4 assays 
Active assay: RNA (29732 features, 0 variable features)
 3 other assays present: ATAC, ATAC_distal, ATAC_promoter

In [4]:
seurat <- readRDS('seurat_with_motif_matrix.rds')  # skip data prep.

In [5]:
seurat <- NormalizeData(seurat, normalization.method = "LogNormalize", assay = "RNA")
seurat <- ScaleData(seurat, do.center = TRUE, do.scale = FALSE)


Centering data matrix



In [6]:
for (i in c("ATAC_distal","ATAC_promoter")) {
  seurat <- RunTFIDF(seurat, assay = i)
}

Performing TF-IDF normalization

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from atac_distal_ to atacdistal_"
Performing TF-IDF normalization

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from atac_promoter_ to atacpromoter_"


In [7]:
seurat <- FindVariableFeatures(seurat, 
  selection.method = "vst", 
  nfeatures = 5000,
  assay = "RNA",
  verbose = FALSE
)

In [8]:
for (i in c("ATAC_distal","ATAC_promoter")) {
  seurat <- FindTopFeatures(seurat, assay=i, min.cutoff = 2000)
  print(length(seurat[[i]]@var.features))
}

[1] 10721
[1] 11646


### save prepped data before proceeding to next step

In [9]:
saveRDS(seurat, "seurat_prepped.rds")

### clear memory, because jupyter kept crashing in next steps

In [10]:
gc()

Warning message:
"'memory.size()' is Windows-specific"


[1] Inf

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,13049195,697.0,23490667,1254.6,15502831,828.0
Vcells,932488373,7114.4,1457284296,11118.2,1439080965,10979.4


Warning message:
"'memory.size()' is Windows-specific"


[1] Inf

### create mofa object

In [11]:
mofa <- create_mofa(seurat, assays = c("RNA","ATAC_distal","ATAC_promoter"))
mofa

Creating MOFA object from a Seurat object...

No features specified, using variable features from the Seurat object...

No groups provided as argument... we assume that all samples are coming from the same group.




Untrained MOFA model with the following characteristics: 
 Number of views: 3 
 Views names: RNA ATAC_distal ATAC_promoter 
 Number of features (per view): 5000 10721 11646 
 Number of groups: 1 
 Groups names: group1 
 Number of samples (per group): 11909 


In [12]:
model_opts <- get_default_model_options(mofa)
model_opts$num_factors <- 15

In [13]:
mofa <- prepare_mofa(mofa,
  model_options = model_opts
)

Warning message in prepare_mofa(mofa, model_options = model_opts):
"Some view(s) have a lot of features, it is recommended to performa more stringent feature selection before creating the MOFA object...."
Checking data options...

No data options specified, using default...

No training options specified, using default...

Checking model options...



### train mofa object (this step takes hours)

In [14]:
# saveRDS(mofa, "mofa.rds")

In [1]:
mofa <- readRDS('mofa.rds') 

In [ ]:
# run `pip install mofapy2==0.5.6` in bash if mofapy2 is not installed
######  mofa <- run_mofa(mofa) # jupyter keep crashing. Use terminal for this step.